In [2]:
# Парсинг КБЖУ продуктов сайта Е-доставка

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import csv
import os

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException

import undetected_chromedriver as uc

import pandas as pd

from tqdm import tqdm

In [3]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [100]:
options = Options()
# options.add_argument('--headless')
# options.add_argument("--disable-infobars")
# options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
# options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(options=options)

for i in range(1, 11): # Проверять количество страниц

    # https://edostavka.by/category/5138?page={i} Овощи и фрукты
    # https://edostavka.by/category/5194?page={i} Молоко и яйца
    # https://edostavka.by/category/5329?page={i} Хлеб и выпечка 
    # https://edostavka.by/category/5131?page={i} Мясо, птица, колбасы
    # https://edostavka.by/category/5185?page={i} Рыба, морепродукты
    # https://edostavka.by/category/5258?page={i} Замороженные продукты
    # https://edostavka.by/category/5150?page={i} Кофе, чай
    # https://edostavka.by/category/5309?page={i} Вода, напитки
    # https://edostavka.by/category/5034?page={i} Крупы, макароны, сахар
    # https://edostavka.by/category/5199?page={i} Масло, консервация, соусы
    # https://edostavka.by/category/5091?page={i} Чипсы, орехи, снеки
    # https://edostavka.by/category/5160?page={i} Сладости
    # https://edostavka.by/category/4996?page={i} Здоровое питание
    # https://edostavka.by/category/4974?page={i} Кулинария
    
    driver.get(f'https://edostavka.by/category/5194?page={i}')
    
    html = driver.find_element(By.TAG_NAME, 'html')
    for n in range(100):
        html.send_keys(Keys.END)
    
    if driver.find_elements(By.XPATH, '/html/body/div[5]/div/div[2]/div/div[2]/button[2]'):
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[2]/div/div[2]/button[2]'))).click()
        for n in range(100):
            html.send_keys(Keys.END)
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'card-image_link__lTrk0')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        with open(f'category_products_page/pages/page_{i}.html', 'w', encoding="utf-8") as file:
            file.write(str(soup))
    else:
        driver.execute_script("window.scrollTo(0, 1080)")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'card-image_link__lTrk0')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        with open(f'category_products_page/pages/page_{i}.html', 'w', encoding="utf-8") as file:
            file.write(str(soup))

driver.quit()

In [101]:
product_links = []
link = 'https://edostavka.by'
for file in os.listdir('category_products_page/pages'):
    soup = BeautifulSoup(open(f'category_products_page/pages/{file}', 'r', encoding="utf-8").read(), 'html.parser')
    for card_div_tag in soup.find_all('div', class_='adult-wrapper_adult__eCCJW vertical_product__Q8mUI'):
        product_link = card_div_tag.find('a', class_='card-image_link__lTrk0').get('href')
        product_link_ready = link + product_link
        product_links.append(product_link_ready)

In [102]:
options = Options()
# options.add_argument('--headless')
# options.add_argument("--disable-infobars")
# options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(options=options)

for product_link in tqdm(product_links):
    product_id = product_link.split('https://edostavka.by/product/')[1]
    if not f'{product_id}.html' in os.listdir('category_products_page/product_pages'):        
        driver.get(product_link)
        if driver.find_elements(By.XPATH, '/html/body/div[5]/div/div[2]/div/div[2]/button[2]'):
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[2]/div/div[2]/button[2]'))).click()
            driver.execute_script("window.scrollTo(0, 700)")
            if driver.find_elements(By.XPATH, '//*[@id="tab_specifications"]'):
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tab_specifications"]'))).click()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            with open(f'category_products_page/product_pages/{product_id}.html', 'w', encoding="utf-8") as file:
                file.write(str(soup))
        else:
            driver.execute_script("window.scrollTo(0, 700)")
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tab_specifications"]'))).click()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            with open(f'category_products_page/product_pages/{product_id}.html', 'w', encoding="utf-8") as file:
                file.write(str(soup))

driver.quit()

100%|██████████| 1404/1404 [11:05<00:00,  2.11it/s]


In [103]:
import re

full_df = pd.DataFrame()

for file in tqdm(os.listdir('category_products_page/product_pages')):
    try:
        soup = BeautifulSoup(open(f'category_products_page/product_pages/{file}', 'r', encoding="utf-8").read(), 'html.parser')
    
        product_name = soup.find('h1', class_='typography typography_tag_h1 typography_weight_700 heading_heading__text__gP6AN heading_heading__text_level_1__7_duQ').text
        product_protein = re.sub(r'[А-я]', ' ', soup.find('div', class_='table_properties__SlSR7').text).split()[0]
        product_fats = re.sub(r'[А-я]', ' ', soup.find('div', class_='table_properties__SlSR7').text).split()[1]
        product_carbohydrate = re.sub(r'[А-я]', ' ', soup.find('div', class_='table_properties__SlSR7').text).split()[2]
        product_calorie = re.sub(r'[А-я]', ' ', soup.find('div', class_='table_properties__SlSR7').text).split()[3]
        
        data = {'name':[product_name], 'protein':[product_protein], 'fats':[product_fats], 'carbohydrate':[product_carbohydrate], 'calorie':[product_calorie]}

        df = pd.DataFrame(data=data)

        full_df = pd.concat([full_df, df])

    except:
        pass

100%|██████████| 1567/1567 [01:29<00:00, 17.53it/s]


In [104]:
full_df.to_csv('edostavka_products.csv', sep=';', index=False)